<a href="https://colab.research.google.com/github/PauloSutti/SP_psy_sessions/blob/main/Modelagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install feature_engine

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import missingno as ms
import matplotlib.pyplot as plt
import re
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.pipeline import Pipeline

In [ ]:
df_reg_tot = pd.read_csv('abt_psi_limpa_regulares_total.csv',sep='|')
df_reg_out = pd.read_csv('abt_psi_limpa_regulares_outliers.csv',sep='|')
df_reg_norm = pd.read_csv('abt_psi_limpa_regulares_normal.csv',sep='|')
df_prop_tot = pd.read_csv('abt_psi_limpa_prop_total.csv',sep='|')
df_prop_out = pd.read_csv('abt_psi_limpa_prop_outliers.csv',sep='|')
df_prop_norm = pd.read_csv('abt_psi_limpa_prop_normal.csv',sep='|')
df_tot = pd.read_csv('abt_psi_limpa_total.csv',sep='|')
df_out = pd.read_csv('abt_psi_limpa_outliers.csv',sep='|')
df_norm = pd.read_csv('abt_psi_limpa_normal.csv',sep='|')

df_reg_tot.name = 'reg_tot'
df_reg_out.name = 'reg_out'
df_reg_norm.name = 'reg_norm'
df_prop_tot.name = 'prop_tot'
df_prop_out.name = 'prop_out'
df_prop_norm.name = 'prop_norm'
df_tot.name = 'tot'
df_out.name = 'out'
df_norm.name = 'norm'

In [ ]:
df_reg_tot.drop('Unnamed: 0', axis = 1, inplace = True)
df_reg_out.drop('Unnamed: 0', axis = 1, inplace = True)
df_reg_norm.drop('Unnamed: 0', axis = 1, inplace = True)
df_prop_tot.drop('Unnamed: 0', axis = 1, inplace = True)
df_prop_out.drop('Unnamed: 0', axis = 1, inplace = True)
df_prop_norm.drop('Unnamed: 0', axis = 1, inplace = True)
df_tot.drop('Unnamed: 0', axis = 1, inplace = True)
df_out.drop('Unnamed: 0', axis = 1, inplace = True)
df_norm.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
ids = ['Code','Município','Cidades','data_ref']
scaler = StandardScaler()
cols_reg = list(set(df_reg_tot.columns) - set(ids))
cols_prop = list(set(df_prop_tot.columns) - set(ids))
cols = list(set(df_tot.columns) - set(ids))

df_reg_tot_std = scaler.fit_transform(df_reg_tot.filter(cols_reg))
df_reg_tot_std = pd.DataFrame(df_reg_tot_std, columns = cols_reg)
for id in ids:
  df_reg_tot_std[id] = df_reg_tot[id]

df_reg_out_std = scaler.fit_transform(df_reg_out.filter(cols_reg))
df_reg_out_std = pd.DataFrame(df_reg_out_std, columns = cols_reg)
for id in ids:
  df_reg_out_std[id] = df_reg_out[id]

df_reg_norm_std = scaler.fit_transform(df_reg_norm.filter(cols_reg))
df_reg_norm_std = pd.DataFrame(df_reg_norm_std, columns = cols_reg)
for id in ids:
  df_reg_norm_std[id] = df_reg_norm[id]

df_prop_tot_std = scaler.fit_transform(df_prop_tot.filter(cols_prop))
df_prop_tot_std = pd.DataFrame(df_prop_tot_std, columns = cols_prop)
for id in ids:
  df_prop_tot_std[id] = df_prop_tot[id]

df_prop_out_std = scaler.fit_transform(df_prop_out.filter(cols_prop))
df_prop_out_std = pd.DataFrame(df_prop_out_std, columns = cols_prop)
for id in ids:
  df_prop_out_std[id] = df_prop_out[id]

df_prop_norm_std = scaler.fit_transform(df_prop_norm.filter(cols_prop))
df_prop_norm_std = pd.DataFrame(df_prop_norm_std, columns = cols_prop)
for id in ids:
  df_prop_norm_std[id] = df_prop_norm[id]

df_tot_std = scaler.fit_transform(df_tot.filter(cols))
df_tot_std = pd.DataFrame(df_tot_std, columns = cols)
for id in ids:
  df_tot_std[id] = df_tot[id]

df_out_std = scaler.fit_transform(df_out.filter(cols))
df_out_std = pd.DataFrame(df_out_std, columns = cols)
for id in ids:
  df_out_std[id] = df_out[id]

df_norm_std = scaler.fit_transform(df_norm.filter(cols))
df_norm_std = pd.DataFrame(df_norm_std, columns = cols)
for id in ids:
  df_norm_std[id] = df_norm[id]

In [ ]:
df_reg_tot_std.name = 'reg_tot_std'
df_reg_out_std.name = 'reg_out_std'
df_reg_norm_std.name = 'reg_norm_std'
df_prop_tot_std.name = 'prop_tot_std'
df_prop_out_std.name = 'prop_out_std'
df_prop_norm_std.name = 'prop_norm_std'
df_tot_std.name = 'tot_std'
df_out_std.name = 'out_std'
df_norm_std.name = 'norm_std'

In [ ]:
bases = [df_reg_tot, df_reg_norm, df_reg_out, df_reg_tot_std, df_reg_norm_std, df_reg_out_std, df_prop_tot, df_prop_norm, df_prop_out, df_prop_tot_std, df_prop_norm_std, df_prop_out_std, df_tot, df_norm, df_out, df_tot_std, df_norm_std, df_out_std]
inputs = []
n = 0
for base in bases:
  if n < 6:
    globals()['X_tr_'+base.name], globals()['X_tst_'+base.name], globals()['y_tr_'+base.name], globals()['y_tst_'+base.name] = train_test_split(base.drop('Psi_qtd_media', axis = 1), base['Psi_qtd_media'], test_size = 0.3, random_state = 42)
    inputs.append(['X_tr_'+base.name, 'X_tst_'+base.name, 'y_tr_'+base.name, 'y_tst_'+base.name])
  elif 5 < n < 12:
    globals()['X_tr_'+base.name], globals()['X_tst_'+base.name], globals()['y_tr_'+base.name], globals()['y_tst_'+base.name] = train_test_split(base.drop('Psi_qtd_media_1000', axis = 1), base['Psi_qtd_media_1000'], test_size = 0.3, random_state = 42)
    inputs.append(['X_tr_'+base.name, 'X_tst_'+base.name, 'y_tr_'+base.name, 'y_tst_'+base.name])
  else:
    globals()['X_tr_1000_'+base.name], globals()['X_tst_1000_'+base.name], globals()['y_tr_1000_'+base.name], globals()['y_tst_1000_'+base.name] = train_test_split(base.drop(['Psi_qtd_media_1000', 'Psi_qtd_media'], axis = 1), base['Psi_qtd_media_1000'], test_size = 0.3, random_state = 42)
    globals()['X_tr_'+base.name], globals()['X_tst_'+base.name], globals()['y_tr_'+base.name], globals()['y_tst_'+base.name] = train_test_split(base.drop(['Psi_qtd_media_1000', 'Psi_qtd_media'], axis = 1), base['Psi_qtd_media'], test_size = 0.3, random_state = 42)
    inputs.append(['X_tr_1000_'+base.name, 'X_tst_1000_'+base.name, 'y_tr_1000_'+base.name, 'y_tst_1000_'+base.name])
    inputs.append(['X_tr_'+base.name, 'X_tst_'+base.name, 'y_tr_'+base.name, 'y_tst_'+base.name])
  n = n + 1

In [ ]:
inputs

[['X_tr_reg_tot', 'X_tst_reg_tot', 'y_tr_reg_tot', 'y_tst_reg_tot'],
 ['X_tr_reg_norm', 'X_tst_reg_norm', 'y_tr_reg_norm', 'y_tst_reg_norm'],
 ['X_tr_reg_out', 'X_tst_reg_out', 'y_tr_reg_out', 'y_tst_reg_out'],
 ['X_tr_reg_tot_std',
  'X_tst_reg_tot_std',
  'y_tr_reg_tot_std',
  'y_tst_reg_tot_std'],
 ['X_tr_reg_norm_std',
  'X_tst_reg_norm_std',
  'y_tr_reg_norm_std',
  'y_tst_reg_norm_std'],
 ['X_tr_reg_out_std',
  'X_tst_reg_out_std',
  'y_tr_reg_out_std',
  'y_tst_reg_out_std'],
 ['X_tr_prop_tot', 'X_tst_prop_tot', 'y_tr_prop_tot', 'y_tst_prop_tot'],
 ['X_tr_prop_norm', 'X_tst_prop_norm', 'y_tr_prop_norm', 'y_tst_prop_norm'],
 ['X_tr_prop_out', 'X_tst_prop_out', 'y_tr_prop_out', 'y_tst_prop_out'],
 ['X_tr_prop_tot_std',
  'X_tst_prop_tot_std',
  'y_tr_prop_tot_std',
  'y_tst_prop_tot_std'],
 ['X_tr_prop_norm_std',
  'X_tst_prop_norm_std',
  'y_tr_prop_norm_std',
  'y_tst_prop_norm_std'],
 ['X_tr_prop_out_std',
  'X_tst_prop_out_std',
  'y_tr_prop_out_std',
  'y_tst_prop_out_std'],


In [ ]:
for n in range(24):
  for s in range(4):
    print(f'{inputs[n][s]}: {globals()[inputs[n][s]].shape}')

X_tr_reg_tot: (7224, 27)
X_tst_reg_tot: (3096, 27)
y_tr_reg_tot: (7224,)
y_tst_reg_tot: (3096,)
X_tr_reg_norm: (5936, 27)
X_tst_reg_norm: (2544, 27)
y_tr_reg_norm: (5936,)
y_tst_reg_norm: (2544,)
X_tr_reg_out: (1288, 27)
X_tst_reg_out: (552, 27)
y_tr_reg_out: (1288,)
y_tst_reg_out: (552,)
X_tr_reg_tot_std: (7224, 27)
X_tst_reg_tot_std: (3096, 27)
y_tr_reg_tot_std: (7224,)
y_tst_reg_tot_std: (3096,)
X_tr_reg_norm_std: (5936, 27)
X_tst_reg_norm_std: (2544, 27)
y_tr_reg_norm_std: (5936,)
y_tst_reg_norm_std: (2544,)
X_tr_reg_out_std: (1288, 27)
X_tst_reg_out_std: (552, 27)
y_tr_reg_out_std: (1288,)
y_tst_reg_out_std: (552,)
X_tr_prop_tot: (7224, 32)
X_tst_prop_tot: (3096, 32)
y_tr_prop_tot: (7224,)
y_tst_prop_tot: (3096,)
X_tr_prop_norm: (5936, 32)
X_tst_prop_norm: (2544, 32)
y_tr_prop_norm: (5936,)
y_tst_prop_norm: (2544,)
X_tr_prop_out: (1288, 32)
X_tst_prop_out: (552, 32)
y_tr_prop_out: (1288,)
y_tst_prop_out: (552,)
X_tr_prop_tot_std: (7224, 32)
X_tst_prop_tot_std: (3096, 32)
y_tr_prop

In [ ]:
for n in range(24):
  for s in range(4):
    globals()[inputs[n][s]].reset_index(drop = True, inplace = True)

#Grupo 1 - Valores absolutos, total de registros

###Valores regulares

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Atendimentos_valor_medio',
           'Assalariados',
           'Renda média (R$)',
           'Contribuição para o PIB estadual %',
           'Estabelecimentos_Atend',
           'Masculino',
           'Estabelecimentos_Total',
           'Estabelecimentos_Publicos',
           'Profissionais_Privados',
           'Profissionais_Total',
           'Pop_Total',
           'Estabelecimentos_Psi',
           'Feminino', '30 a 59 anos',
           '0 a 14 anos',
           '60+ anos',
           'Profissionais_Publicos',
           'Atendimentos_qtd_media',
           'Estabelecimentos_Privados',
           '15 a 29 anos']

cols = list((set(X_tr_reg_tot.columns) - set(ids)) - set(removed))
X = X_tr_reg_tot.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
2,Renda média em Salários mínimos,1.065143
1,PIB (mil R$),2.278619
0,Psi_valor_medio,2.364638


In [ ]:
cols = list((set(X_tr_reg_tot.columns) - set(ids)) - set(removed))
X = X_tr_reg_tot.filter(cols).copy()

X = sm.add_constant(X)

model_reg_tot = sm.OLS(y_tr_reg_tot, X).fit()

model_reg_tot.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Psi_qtd_media   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.875
Method:                 Least Squares   F-statistic:                 1.683e+04
Date:                Sun, 24 Oct 2021   Prob (F-statistic):               0.00
Time:                        21:43:39   Log-Likelihood:                -64037.
No. Observations:                7224   AIC:                         1.281e+05
Df Residuals:                    7220   BIC:                         1.281e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                             280.0932     74.305      3.770      0.000     134.434     425.753
Psi_valor_medio                     0.0988      0.001     78.661      0.000       0.096       0.101
PIB (mil R$)                        0.0001    1.3e-06     81.498      0.000       0.000       0.000
Renda média em Salários mínimos  -114.5921     29.073     -3.941      0.000    -171.584     -57.600
==============================================================================
Omnibus:                     5286.873   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         35676497.547
Skew:                          -1.750   Prob(JB):                         0.00
Kurtosis:                     347.259   Cond. No.                     9.24e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.24e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
3,Renda média em Salários mínimos,1.043755
2,PIB (mil R$),2.265469
1,Psi_valor_medio,2.273828
0,const,13.597536


In [ ]:
#Intercepto removido por indicativo de multicolinearidade
#Variável de Renda média em Salários mínimos removida por p-valor acima do limite escolhido (confiança de 0.95)
removed.append('Renda média em Salários mínimos')

cols = list((set(X_tr_reg_tot.columns) - set(ids)) - set(removed))
X = X_tr_reg_tot.filter(cols).copy()

model_reg_tot = sm.OLS(y_tr_reg_tot, X).fit()

model_reg_tot.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Psi_qtd_media   R-squared (uncentered):                   0.878
Model:                            OLS   Adj. R-squared (uncentered):              0.877
Method:                 Least Squares   F-statistic:                          2.587e+04
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:43:44   Log-Likelihood:                         -64045.
No. Observations:                7224   AIC:                                  1.281e+05
Df Residuals:                    7222   BIC:                                  1.281e+05
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Psi_valor_medio     0.0983      0.001     79.819      0.000       0.096       0.101
PIB (mil R$)        0.0001    1.3e-06     81.499      0.000       0.000       0.000
==============================================================================
Omnibus:                     5200.391   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         35244279.961
Skew:                          -1.683   Prob(JB):                         0.00
Kurtosis:                     345.169   Cond. No.                     1.43e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
predict_tst = model_reg_tot.predict(X_tst_reg_tot.filter(cols))
predict_tr = model_reg_tot.predict(X_tr_reg_tot.filter(cols))
r2_score(y_tst_reg_tot, predict_tst)

0.8031800894399642

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred)/y_true))

In [ ]:
mape(y_tr_reg_tot, predict_tr), mape(y_tst_reg_tot, predict_tst)

(inf, inf)

Para o grupo 1 (base com variáveis de valores absolutos e total de registros), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.877 em treino e 0.803 em teste. Porém, observando a métrica MAPE, temos resultados bastante insatisfatórios.

Foram utilizadas apenas as variáveis:
1. PIB (mil R$)
2. Psi_valor_médio

###Valores Normalizados

In [ ]:
removed = ['Profissionais_Privados',
           'Feminino','0 a 14 anos',
           'Estabelecimentos_Privados',
           'Pop_Total','Masculino',
           '30 a 59 anos',
           'Estabelecimentos_Atend',
           '60+ anos',
           'Assalariados',
           'Contribuição para o PIB estadual %',
           'Profissionais_Total',
           'Profissionais_Publicos',
           'Atendimentos_qtd_media',
           'Estabelecimentos_Psi',
           'Estabelecimentos_Publicos',
           'Atendimentos_valor_medio',
           'Estabelecimentos_Total',
           'PIB (mil R$)']

cols = list((set(X_tr_reg_tot_std.columns) - set(ids)) - set(removed))
X = X_tr_reg_tot_std.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
1,Renda média (R$),1.290952
2,Renda média em Salários mínimos,1.300426
0,15 a 29 anos,2.263322
3,Psi_valor_medio,2.285153


In [ ]:
cols = list((set(X_tr_reg_tot_std.columns) - set(ids)) - set(removed))
X = X_tr_reg_tot_std.filter(cols).copy()

#intercepto excluído após constatação de p-valor de 0.744
#X = sm.add_constant(X)

model_reg_tot_std = sm.OLS(y_tr_reg_tot_std, X).fit()

model_reg_tot_std.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Psi_qtd_media   R-squared (uncentered):                   0.804
Model:                            OLS   Adj. R-squared (uncentered):              0.804
Method:                 Least Squares   F-statistic:                              7427.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:05   Log-Likelihood:                         -4738.6
No. Observations:                7224   AIC:                                      9485.
Df Residuals:                    7220   BIC:                                      9513.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
15 a 29 anos                        0.3197      0.008     39.958      0.000       0.304       0.335
Renda média (R$)                    0.0610      0.006      9.867      0.000       0.049       0.073
Renda média em Salários mínimos    -0.0402      0.006     -6.652      0.000      -0.052      -0.028
Psi_valor_medio                     0.6509      0.008     80.758      0.000       0.635       0.667
==============================================================================
Omnibus:                     6518.664   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         21229875.590
Skew:                           2.914   Prob(JB):                         0.00
Kurtosis:                     268.513   Cond. No.                         2.81
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
predict_tst = model_reg_tot_std.predict(X_tst_reg_tot_std.filter(cols))
predict_tr = model_reg_tot_std.predict(X_tr_reg_tot_std.filter(cols))
r2_score(y_tst_reg_tot_std, predict_tst)

0.7514404985368204

In [ ]:
mape(y_tr_reg_tot_std, predict_tr), mape(y_tst_reg_tot_std, predict_tst)

(1.1104805207981399, 0.9539281933998437)

Para o grupo 1 (base com variáveis de valores normalizados e total de registros), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.804 em treino e 0.751 em teste. Porém, observando a métrica MAPE, temos resultados bastante insatisfatórios - ainda que melhores que em modelagem com valores absolutos.

Foram utilizadas apenas as variáveis:
1. Renda média (R$)
2. Renda média em Salários mínimos
3. Psi_valor_médio
4. 15 a 29 anos

#Grupo 2 - valores absolutos, bases divididas entre outliers (b) e restante (a)

###Base a., Valores Regulares

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Feminino',
           'Estabelecimentos_Privados',
           'Profissionais_Publicos',
           '0 a 14 anos',
           'Pop_Total',
           'Masculino',
           '30 a 59 anos',
           'Estabelecimentos_Atend',
           'Contribuição para o PIB estadual %',
           '15 a 29 anos','60+ anos',
           'Estabelecimentos_Psi',
           'Renda média (R$)',
           'Profissionais_Total',
           'Estabelecimentos_Publicos',
           'Assalariados']
cols = list((set(X_tr_reg_norm.columns) - set(ids)) - set(removed))
X = X_tr_reg_norm.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
1,Psi_valor_medio,1.169428
0,PIB (mil R$),1.851144
5,Renda média em Salários mínimos,2.005265
3,Profissionais_Privados,2.061640
6,Atendimentos_valor_medio,2.368428
4,Atendimentos_qtd_media,3.089791
2,Estabelecimentos_Total,3.771688


In [ ]:
removed.append('Renda média em Salários mínimos')
removed.append('Atendimentos_valor_medio')
removed.append('Estabelecimentos_Total')
removed.append('PIB (mil R$)')

cols = list((set(X_tr_reg_norm.columns) - set(ids)) - set(removed))
X = X_tr_reg_norm.filter(cols).copy()

#intercepto excluído
#X = sm.add_constant(X)

model_reg_norm = sm.OLS(y_tr_reg_norm, X).fit()

model_reg_norm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Psi_qtd_media   R-squared (uncentered):                   0.379
Model:                            OLS   Adj. R-squared (uncentered):              0.378
Method:                 Least Squares   F-statistic:                              1206.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:08   Log-Likelihood:                         -47029.
No. Observations:                5936   AIC:                                  9.406e+04
Df Residuals:                    5933   BIC:                                  9.408e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Psi_valor_medio            0.0969      0.002     52.927      0.000       0.093       0.100
Atendimentos_qtd_media     0.0028      0.000      7.042      0.000       0.002       0.004
Profissionais_Privados    12.9756      3.551      3.654      0.000       6.014      19.938
==============================================================================
Omnibus:                    15062.977   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        212269477.662
Skew:                          28.259   Prob(JB):                         0.00
Kurtosis:                     927.683   Cond. No.                     1.06e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.06e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Para o grupo 2.a. (base com variáveis de valores absolutos e cidades com baixa concentração de outliers), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.378 em treino - baixa explicabilidade da variável target.

Foram utilizadas apenas as variáveis:
1. Profissionais_Privados
2. Psi_valor_medio
3. Atendimentos_qtd_media

###Base a., Valores Normalizados

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Profissionais_Privados',
           'Feminino',
           '0 a 14 anos',
           'Estabelecimentos_Privados',
           'Pop_Total',
           'Masculino',
           '30 a 59 anos',
           'Estabelecimentos_Atend',
           'Contribuição para o PIB estadual %',
           '15 a 29 anos',
           'Profissionais_Total',
           '60+ anos',
           'Estabelecimentos_Total']
cols = list((set(X_tr_reg_norm_std.columns) - set(ids)) - set(removed))
X = X_tr_reg_norm_std.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
3,Psi_valor_medio,1.191593
8,Renda média em Salários mínimos,1.398262
7,Renda média (R$),1.512865
4,Atendimentos_valor_medio,1.934004
5,Atendimentos_qtd_media,1.984084
0,PIB (mil R$),2.237441
6,Profissionais_Publicos,2.268748
9,Estabelecimentos_Psi,2.834687
1,Estabelecimentos_Publicos,3.717933
2,Assalariados,4.263060


In [ ]:
removed.append('Estabelecimentos_Publicos')
removed.append('PIB (mil R$)')
removed.append('Estabelecimentos_Psi')
removed.append('Renda média (R$)')
removed.append('Renda média em Salários mínimos')

cols = list((set(X_tr_reg_norm_std.columns) - set(ids)) - set(removed))
X = X_tr_reg_norm_std.filter(cols).copy()

#intercepto excluído
#X = sm.add_constant(X)

model_reg_norm_std = sm.OLS(y_tr_reg_norm_std, X).fit()

model_reg_norm_std.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Psi_qtd_media   R-squared (uncentered):                   0.383
Model:                            OLS   Adj. R-squared (uncentered):              0.382
Method:                 Least Squares   F-statistic:                              734.9
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:10   Log-Likelihood:                         -7684.5
No. Observations:                5936   AIC:                                  1.538e+04
Df Residuals:                    5931   BIC:                                  1.541e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Assalariados                -0.0565      0.016     -3.520      0.000      -0.088      -0.025
Psi_valor_medio              0.6085      0.013     46.437      0.000       0.583       0.634
Atendimentos_qtd_media       0.0346      0.016      2.202      0.028       0.004       0.065
Profissionais_Publicos       0.2675      0.016     16.716      0.000       0.236       0.299
Atendimentos_valor_medio    -0.0588      0.016     -3.776      0.000      -0.089      -0.028
==============================================================================
Omnibus:                    14954.200   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        204370779.068
Skew:                          27.741   Prob(JB):                         0.00
Kurtosis:                     910.314   Cond. No.                         2.82
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Para o grupo 2.a. (base com variáveis de valores normalizados e cidades com baixa concentração de outliers), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.382 em treino - baixa explicabilidade da variável target.

Foram utilizadas apenas as variáveis:
1. Atendimentos_valor_medio
2. Atendimentos_qtd_media
3. Profissionais_Publicos
4. Assalariados
5. Psi_valor_medio

###Base b., Valores Absolutos

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Profissionais_Privados',
           'Feminino',
           '0 a 14 anos',
           'Estabelecimentos_Privados',
           'Pop_Total',
           'Masculino',
           '30 a 59 anos',
           'Estabelecimentos_Atend',
           'Assalariados',
           '60+ anos',
           '15 a 29 anos',
           'Profissionais_Total',
           'Profissionais_Publicos',
           'Atendimentos_qtd_media',
           'Estabelecimentos_Publicos',
           'Estabelecimentos_Psi',
           'Atendimentos_valor_medio',
           'Estabelecimentos_Total',
           'Contribuição para o PIB estadual %',
           'Renda média (R$)']
cols = list((set(X_tr_reg_out.columns) - set(ids)) - set(removed))
X = X_tr_reg_out.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
2,Renda média em Salários mínimos,1.225326
1,PIB (mil R$),2.443677
0,Psi_valor_medio,2.752741


In [ ]:
cols = list((set(X_tr_reg_out.columns) - set(ids)) - set(removed))
X = X_tr_reg_out.filter(cols).copy()

X = sm.add_constant(X)

model_reg_out = sm.OLS(y_tr_reg_out, X).fit()

model_reg_out.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Psi_qtd_media   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                     3782.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):               0.00
Time:                        21:44:11   Log-Likelihood:                -12337.
No. Observations:                1288   AIC:                         2.468e+04
Df Residuals:                    1284   BIC:                         2.470e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                            1884.1130    426.637      4.416      0.000    1047.131    2721.095
Psi_valor_medio                     0.0803      0.003     27.775      0.000       0.075       0.086
PIB (mil R$)                        0.0001   2.67e-06     45.241      0.000       0.000       0.000
Renda média em Salários mínimos  -576.6552    135.386     -4.259      0.000    -842.256    -311.054
==============================================================================
Omnibus:                      804.750   Durbin-Watson:                   2.048
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           243655.112
Skew:                          -1.696   Prob(JB):                         0.00
Kurtosis:                      70.295   Cond. No.                     2.70e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.7e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
3,Renda média em Salários mínimos,1.070021
1,Psi_valor_medio,2.374089
2,PIB (mil R$),2.478104
0,const,19.107096


In [ ]:
removed.append('Renda média em Salários mínimos')

cols = list((set(X_tr_reg_out.columns) - set(ids)) - set(removed))
X = X_tr_reg_out.filter(cols).copy()

#intercepto excluído
#X = sm.add_constant(X)

model_reg_out = sm.OLS(y_tr_reg_out, X).fit()

model_reg_out.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Psi_qtd_media   R-squared (uncentered):                   0.906
Model:                            OLS   Adj. R-squared (uncentered):              0.906
Method:                 Least Squares   F-statistic:                              6232.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:12   Log-Likelihood:                         -12347.
No. Observations:                1288   AIC:                                  2.470e+04
Df Residuals:                    1286   BIC:                                  2.471e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Psi_valor_medio     0.0825      0.003     30.660      0.000       0.077       0.088
PIB (mil R$)        0.0001   2.59e-06     45.446      0.000       0.000       0.000
==============================================================================
Omnibus:                      853.553   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           234263.992
Skew:                          -1.922   Prob(JB):                         0.00
Kurtosis:                      68.958   Cond. No.                     1.62e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
predict_tst = model_reg_out.predict(X_tst_reg_out.filter(cols))
predict_tr = model_reg_out.predict(X_tr_reg_out.filter(cols))
r2_score(y_tst_reg_out, predict_tst)

0.6866402097608963

In [ ]:
mape(y_tr_reg_out, predict_tr), mape(y_tst_reg_out, predict_tst)

(inf, inf)

Para o grupo 2.b. (base com variáveis de valores absolutos e cidades com alta concentração de outliers), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.906 em treino e 0.686 em teste. Porém, observando a métrica MAPE, temos resultados bastante insatisfatórios.

Foram utilizadas apenas as variáveis:
1. Psi_valor_medio
2. PIB (mil R$)

###Base b., Valores Normalizados

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Profissionais_Privados',
           'Feminino',
           '0 a 14 anos',
           'Estabelecimentos_Privados',
           'Pop_Total',
           'Masculino',
           '30 a 59 anos',
           'Estabelecimentos_Atend',
           'Assalariados',
           '60+ anos',
           '15 a 29 anos',
           'Profissionais_Total',
           'Profissionais_Publicos',
           'Contribuição para o PIB estadual %',
           'Estabelecimentos_Publicos',
           'Atendimentos_qtd_media',
           'Estabelecimentos_Psi',
           'Atendimentos_valor_medio',
           'Estabelecimentos_Total']
cols = list((set(X_tr_reg_out_std.columns) - set(ids)) - set(removed))
X = X_tr_reg_out_std.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
1,Renda média (R$),1.141378
3,Renda média em Salários mínimos,1.143948
0,Psi_valor_medio,2.374408
2,PIB (mil R$),2.509757


In [ ]:
cols = list((set(X_tr_reg_out_std.columns) - set(ids)) - set(removed))
X = X_tr_reg_out_std.filter(cols).copy()

#intercepto excluído após constatação de p-valor de 0.627
#X = sm.add_constant(X)

model_reg_out_std = sm.OLS(y_tr_reg_out_std, X).fit()

model_reg_out_std.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Psi_qtd_media   R-squared (uncentered):                   0.902
Model:                            OLS   Adj. R-squared (uncentered):              0.902
Method:                 Least Squares   F-statistic:                              2966.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:14   Log-Likelihood:                         -406.46
No. Observations:                1288   AIC:                                      820.9
Df Residuals:                    1284   BIC:                                      841.6
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Psi_valor_medio                     0.3936      0.014     28.203      0.000       0.366       0.421
Renda média (R$)                    0.0714      0.010      7.262      0.000       0.052       0.091
PIB (mil R$)                        0.5953      0.013     45.038      0.000       0.569       0.621
Renda média em Salários mínimos    -0.0599      0.010     -6.046      0.000      -0.079      -0.040
==============================================================================
Omnibus:                      798.851   Durbin-Watson:                   2.072
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           226982.702
Skew:                          -1.688   Prob(JB):                         0.00
Kurtosis:                      67.947   Cond. No.                         2.91
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
predict_tst = model_reg_out_std.predict(X_tst_reg_out_std.filter(cols))
predict_tr = model_reg_out_std.predict(X_tr_reg_out_std.filter(cols))
r2_score(y_tst_reg_out_std, predict_tst)

0.6860327724915596

In [ ]:
mape(y_tr_reg_out_std, predict_tr), mape(y_tst_reg_out_std, predict_tst)

(1.3318425650227768, 1.401304997728179)

Para o grupo 2.b. (base com variáveis de valores normalizados e cidades com alta concentração de outliers), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.902 em treino e 0.686 em teste. Porém, observando a métrica MAPE, temos resultados bastante insatisfatórios - ainda que melhores que a modelagem com valores absolutos.

Foram utilizadas apenas as variáveis:
1. Renda média (reais)
2. Renda média em Salários mínimos
3. Psi_valor_medio
4. PIB (mil R$)

#Grupo 3 - Valores proporcionais, total de registros

###Valores proporcionais

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Masculino_1000',
           'Estabelecimentos_Total_1000',
           'Profissionais_Privados_1000',
           '30 a 59 anos_1000',
           'Feminino_1000',
           'Estabelecimentos_Publicos_1000',
           'Estabelecimentos_Publicos_Prop',
           '15 a 29 anos_1000',
           'Profissionais_Total_1000',
           'Renda média em Salários mínimos',
           '0 a 14 anos_1000',
           'Renda média (R$)',
           'Estabelecimentos_Privados_Prop',
           '60+ anos_1000',
           'Estabelecimentos_Atend_Prop',
           'Estabelecimentos_Atend_1000',
           'Estabelecimentos_Psi_Prop',
           'Estabelecimentos_Psi_1000']
cols = list((set(X_tr_prop_tot.columns) - set(ids)) - set(removed))
X = X_tr_prop_tot.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
5,Contribuição para o PIB estadual %,1.042695
4,Psi_valor_medio_1000,1.184694
9,Atendimentos_qtd_media_1000,1.620167
7,Atendimentos_valor_medio_1000,1.691171
2,Profissionais_Privados_Prop,2.231890
6,Estabelecimentos_Privados_1000,2.386785
8,PIB (mil R$)_1000,2.868801
0,Profissionais_Publicos_1000,2.946444
1,Assalariados_1000,3.703407
3,Profissionais_Publicos_Prop,3.835932


In [ ]:
removed.append('Contribuição para o PIB estadual %')
removed.append('Atendimentos_valor_medio_1000')
removed.append('Estabelecimentos_Privados_1000')
removed.append('Profissionais_Publicos_Prop')
removed.append('Assalariados_1000')
removed.append('PIB (mil R$)_1000')

cols = list((set(X_tr_prop_tot.columns) - set(ids)) - set(removed))
X = X_tr_prop_tot.filter(cols).copy()

#intercepto excluído após constatação de p-valor de 0.627
#X = sm.add_constant(X)

model_prop_tot = sm.OLS(y_tr_prop_tot, X).fit()

model_prop_tot.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Psi_qtd_media_1000   R-squared (uncentered):                   0.455
Model:                            OLS   Adj. R-squared (uncentered):              0.455
Method:                 Least Squares   F-statistic:                              1509.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:16   Log-Likelihood:                         -32274.
No. Observations:                7224   AIC:                                  6.456e+04
Df Residuals:                    7220   BIC:                                  6.458e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Atendimentos_qtd_media_1000     0.0004      0.000      2.398      0.016    6.68e-05       0.001
Profissionais_Publicos_1000    11.6079      1.082     10.731      0.000       9.487      13.728
Profissionais_Privados_Prop     4.6013      0.889      5.177      0.000       2.859       6.344
Psi_valor_medio_1000            0.0860      0.001     66.467      0.000       0.083       0.089
==============================================================================
Omnibus:                    18959.761   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        410610239.709
Skew:                          30.730   Prob(JB):                         0.00
Kurtosis:                    1169.352   Cond. No.                     8.69e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 8.69e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Para o grupo 3 (base com variáveis de valores proporcionais e total de registros), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.455 em treino - baixa explicabilidade da variável target.

Foram utilizadas apenas as variáveis:
1. Profissionais_Publicos_1000
2. Atendimentos_qtd_media_1000
3. Psi_valor_medio_1000
4. Profissionais_Privados_Prop

###Valores Normalizados

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Masculino_1000',
           'Estabelecimentos_Total_1000',
           '60+ anos_1000',
           'Profissionais_Privados_1000',
           'Estabelecimentos_Privados_Prop',
           'Profissionais_Total_1000',
           '30 a 59 anos_1000',
           'Estabelecimentos_Atend_Prop',
           'Estabelecimentos_Atend_1000']
cols = list((set(X_tr_prop_tot_std.columns) - set(ids)) - set(removed))
X = X_tr_prop_tot_std.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
18,Atendimentos_qtd_media_1000,1.041176
11,Contribuição para o PIB estadual %,1.072022
2,Psi_valor_medio_1000,1.115112
13,Atendimentos_valor_medio_1000,1.280425
7,Estabelecimentos_Psi_Prop,1.319186
5,Assalariados_1000,1.508191
8,Profissionais_Publicos_1000,1.738857
4,PIB (mil R$)_1000,1.825991
16,Feminino_1000,1.996325
9,Profissionais_Publicos_Prop,1.998438


In [ ]:
removed.append('Contribuição para o PIB estadual %')
removed.append('15 a 29 anos_1000')
removed.append('PIB (mil R$)_1000')
removed.append('Profissionais_Privados_Prop')
removed.append('Atendimentos_valor_medio_1000')
removed.append('Feminino_1000')
removed.append('Renda média em Salários mínimos')
removed.append('Estabelecimentos_Publicos_Prop')
removed.append('Assalariados_1000')
removed.append('0 a 14 anos_1000')
removed.append('Profissionais_Publicos_Prop')

cols = list((set(X_tr_prop_tot_std.columns) - set(ids)) - set(removed))
X = X_tr_prop_tot_std.filter(cols).copy()

#intercepto excluído após constatação de p-valor de 0.856
#X = sm.add_constant(X)

model_prop_tot_std = sm.OLS(y_tr_prop_tot_std, X).fit()

model_prop_tot_std.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Psi_qtd_media_1000   R-squared (uncentered):                   0.404
Model:                            OLS   Adj. R-squared (uncentered):              0.403
Method:                 Least Squares   F-statistic:                              611.3
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:17   Log-Likelihood:                         -8487.3
No. Observations:                7224   AIC:                                  1.699e+04
Df Residuals:                    7216   BIC:                                  1.705e+04
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Estabelecimentos_Psi_Prop         -0.0269      0.010     -2.694      0.007      -0.046      -0.007
Estabelecimentos_Psi_1000          0.1026      0.016      6.515      0.000       0.072       0.133
Profissionais_Publicos_1000        0.0720      0.010      7.241      0.000       0.052       0.091
Psi_valor_medio_1000               0.6186      0.010     64.125      0.000       0.600       0.638
Estabelecimentos_Publicos_1000    -0.0440      0.011     -4.001      0.000      -0.066      -0.022
Renda média (R$)                   0.0201      0.010      1.991      0.047       0.000       0.040
Estabelecimentos_Privados_1000    -0.0817      0.015     -5.329      0.000      -0.112      -0.052
Atendimentos_qtd_media_1000        0.0238      0.010      2.319      0.020       0.004       0.044
==============================================================================
Omnibus:                    19019.300   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        419676421.198
Skew:                          30.978   Prob(JB):                         0.00
Kurtosis:                    1182.167   Cond. No.                         3.30
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Para o grupo 3 (base com variáveis de valores normalizados e total de registros), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.403 em treino - baixa explicabilidade da variável target.

Foram utilizadas as variáveis:
1. Estabelecimentos_Psi_Prop
2. Estabelecimentos_Psi_1000
3. Profissionais_Publicos_1000
4. Psi_valor_medio_1000
5. Estabelecimentos_Publicos_1000
6. Renda média (R$)
7. Estabelecimentos_Privados_1000
8. Atendimentos_qtd_media_1000

#Grupo 4 - Valores proporcionais, bases divididas entre outliers (b) e restante (a)

###Base a., Valores proporcionais

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Masculino_1000',
           'Estabelecimentos_Total_1000',
           'Profissionais_Privados_1000',
           '30 a 59 anos_1000',
           'Estabelecimentos_Publicos_Prop',
           'Feminino_1000',
           '15 a 29 anos_1000',
           'Profissionais_Total_1000',
           'Renda média em Salários mínimos',
           '60+ anos_1000',
           '0 a 14 anos_1000',
           'Renda média (R$)',
           'Estabelecimentos_Privados_Prop',
           'Estabelecimentos_Atend_Prop',
           'Estabelecimentos_Atend_1000',
           'Estabelecimentos_Psi_Prop',
           'Estabelecimentos_Psi_1000']
cols = list((set(X_tr_prop_norm.columns) - set(ids)) - set(removed))
X = X_tr_prop_norm.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
4,Psi_valor_medio_1000,1.134769
10,Atendimentos_qtd_media_1000,1.710512
8,Atendimentos_valor_medio_1000,1.722798
2,Profissionais_Privados_Prop,1.882334
7,Estabelecimentos_Privados_1000,2.215400
6,Contribuição para o PIB estadual %,3.176332
1,Assalariados_1000,3.293168
0,Profissionais_Publicos_1000,3.386726
5,Estabelecimentos_Publicos_1000,3.773419
3,Profissionais_Publicos_Prop,4.621946


In [ ]:
removed.append('PIB (mil R$)_1000')
removed.append('Profissionais_Privados_Prop')
removed.append('Atendimentos_qtd_media_1000')
removed.append('Atendimentos_valor_medio_1000')
removed.append('Estabelecimentos_Privados_1000')
removed.append('Estabelecimentos_Publicos_1000')

cols = list((set(X_tr_prop_norm.columns) - set(ids)) - set(removed))
X = X_tr_prop_norm.filter(cols).copy()

X = sm.add_constant(X)

model_prop_norm = sm.OLS(y_tr_prop_norm, X).fit()

model_prop_norm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Psi_qtd_media_1000   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     540.5
Date:                Sun, 24 Oct 2021   Prob (F-statistic):               0.00
Time:                        21:44:19   Log-Likelihood:                -27696.
No. Observations:                5936   AIC:                         5.540e+04
Df Residuals:                    5930   BIC:                         5.545e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                  2.4718      0.971      2.546      0.011       0.569       4.375
Profissionais_Publicos_1000           15.0516      1.991      7.561      0.000      11.149      18.954
Assalariados_1000                     -0.0060      0.003     -2.359      0.018      -0.011      -0.001
Profissionais_Publicos_Prop           -2.3958      1.122     -2.136      0.033      -4.595      -0.197
Psi_valor_medio_1000                   0.0997      0.002     49.675      0.000       0.096       0.104
Contribuição para o PIB estadual %    20.1206      9.222      2.182      0.029       2.041      38.200
==============================================================================
Omnibus:                    15130.616   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        223075311.609
Skew:                          28.567   Prob(JB):                         0.00
Kurtosis:                     950.975   Cond. No.                     6.33e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.33e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
4,Psi_valor_medio_1000,1.028662
2,Assalariados_1000,1.094006
5,Contribuição para o PIB estadual %,1.100817
3,Profissionais_Publicos_Prop,1.251335
1,Profissionais_Publicos_1000,1.325466
0,const,8.454354


In [ ]:
removed.append('Profissionais_Publicos_Prop')
removed.append('Assalariados_1000')

cols = list((set(X_tr_prop_norm.columns) - set(ids)) - set(removed))
X = X_tr_prop_norm.filter(cols).copy()

model_prop_norm = sm.OLS(y_tr_prop_norm, X).fit()

model_prop_norm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Psi_qtd_media_1000   R-squared (uncentered):                   0.349
Model:                            OLS   Adj. R-squared (uncentered):              0.349
Method:                 Least Squares   F-statistic:                              1060.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:19   Log-Likelihood:                         -27701.
No. Observations:                5936   AIC:                                  5.541e+04
Df Residuals:                    5933   BIC:                                  5.543e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Profissionais_Publicos_1000           12.9005      1.235     10.442      0.000      10.479      15.322
Contribuição para o PIB estadual %    16.5769      8.039      2.062      0.039       0.818      32.335
Psi_valor_medio_1000                   0.0999      0.002     49.761      0.000       0.096       0.104
==============================================================================
Omnibus:                    15133.232   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        223263513.227
Skew:                          28.579   Prob(JB):                         0.00
Kurtosis:                     951.375   Cond. No.                     4.19e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 4.19e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Para o grupo 4.a. (base com variáveis de valores proporcionais e cidades com baixa concentração de outliers), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.349 em treino - baixa explicabilidade da variável target.

Foram utilizadas apenas as variáveis:
1. Profissionais_Publicos_1000
2. Contribuição para o PIB estadual %
3. Psi_valor_medio_1000


###Base a., Valores Normalizados



In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Masculino_1000',
           'Estabelecimentos_Total_1000',
           '60+ anos_1000',
           'Profissionais_Privados_1000',
           'Estabelecimentos_Privados_Prop',
           'Profissionais_Publicos_1000',
           '30 a 59 anos_1000',
           'Estabelecimentos_Atend_Prop',
           'Estabelecimentos_Atend_1000',
           'Estabelecimentos_Publicos_Prop']
cols = list((set(X_tr_prop_norm_std.columns) - set(ids)) - set(removed))
X = X_tr_prop_norm_std.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
17,Atendimentos_qtd_media_1000,1.039932
7,Psi_valor_medio_1000,1.088496
15,Atendimentos_valor_medio_1000,1.181116
3,Estabelecimentos_Psi_Prop,1.209736
5,Assalariados_1000,1.417037
6,Profissionais_Publicos_Prop,1.683561
2,Profissionais_Total_1000,1.703651
14,Renda média em Salários mínimos,1.818456
10,Estabelecimentos_Publicos_1000,1.851706
9,Feminino_1000,2.008818


In [ ]:
removed.append('Feminino_1000')
removed.append('15 a 29 anos_1000')
removed.append('0 a 14 anos_1000')
removed.append('Atendimentos_valor_medio_1000')
removed.append('Assalariados_1000')
removed.append('Profissionais_Publicos_Prop')
removed.append('Profissionais_Privados_Prop')
removed.append('Atendimentos_qtd_media_1000')
removed.append('Renda média em Salários mínimos')
removed.append('Renda média (R$)')
removed.append('PIB (mil R$)_1000')
removed.append('Contribuição para o PIB estadual %')
removed.append('Estabelecimentos_Psi_Prop')

cols = list((set(X_tr_prop_norm.columns) - set(ids)) - set(removed))
X = X_tr_prop_norm.filter(cols).copy()

#intercepto removido com p-valor de 0.255
#X = sm.add_constant(X)

model_prop_norm = sm.OLS(y_tr_prop_norm, X).fit()

model_prop_norm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Psi_qtd_media_1000   R-squared (uncentered):                   0.351
Model:                            OLS   Adj. R-squared (uncentered):              0.350
Method:                 Least Squares   F-statistic:                              641.4
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:21   Log-Likelihood:                         -27692.
No. Observations:                5936   AIC:                                  5.539e+04
Df Residuals:                    5931   BIC:                                  5.543e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Estabelecimentos_Psi_1000          3.6070      0.880      4.099      0.000       1.882       5.332
Psi_valor_medio_1000               0.0996      0.002     49.246      0.000       0.096       0.104
Estabelecimentos_Publicos_1000    -2.5251      0.921     -2.742      0.006      -4.331      -0.720
Estabelecimentos_Privados_1000    -5.3612      1.322     -4.054      0.000      -7.953      -2.769
Profissionais_Total_1000          11.8427      1.721      6.882      0.000       8.469      15.216
==============================================================================
Omnibus:                    15131.714   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        223337147.378
Skew:                          28.572   Prob(JB):                         0.00
Kurtosis:                     951.533   Cond. No.                         927.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Para o grupo 4.a. (base com variáveis de valores normalizados e cidades com baixa concentração de outliers), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.35 em treino - baixa explicabilidade da variável target.

Foram utilizadas apenas as variáveis:
1. Estabelecimentos_Psi_1000
2. Psi_valor_medio_1000
3. Estabelecimentos_Publicos_1000
4. Estabelecimentos_Privados_1000
5. Profissionais_Total_1000

###Base b., Valores Proporcionais

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Masculino_1000',
           'Estabelecimentos_Total_1000',
           '60+ anos_1000',
           'Estabelecimentos_Publicos_Prop',
           'Profissionais_Privados_1000',
           'Profissionais_Publicos_Prop',
           'Feminino_1000',
           '15 a 29 anos_1000',
           '30 a 59 anos_1000',
           'Estabelecimentos_Privados_Prop',
           'Renda média em Salários mínimos',
           'Profissionais_Total_1000',
           'Renda média (R$)',
           'Assalariados_1000',
           'Estabelecimentos_Publicos_1000',
           '0 a 14 anos_1000',
           'Profissionais_Publicos_1000',
           'Estabelecimentos_Psi_Prop',
           'Estabelecimentos_Atend_1000',
           'Estabelecimentos_Atend_Prop',
           'Estabelecimentos_Psi_1000']
cols = list((set(X_tr_prop_out.columns) - set(ids)) - set(removed))
X = X_tr_prop_out.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
2,Contribuição para o PIB estadual %,1.071127
1,Psi_valor_medio_1000,1.419399
4,Atendimentos_valor_medio_1000,2.079913
5,PIB (mil R$)_1000,2.737514
3,Estabelecimentos_Privados_1000,3.028036
6,Atendimentos_qtd_media_1000,3.991367
0,Profissionais_Privados_Prop,4.473073


In [ ]:
removed.append('Contribuição para o PIB estadual %')

cols = list((set(X_tr_prop_out.columns) - set(ids)) - set(removed))
X = X_tr_prop_out.filter(cols).copy()

#intercepto removido com p-valor de 0.07
#X = sm.add_constant(X)

model_prop_out = sm.OLS(y_tr_prop_out, X).fit()

model_prop_out.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Psi_qtd_media_1000   R-squared (uncentered):                   0.857
Model:                            OLS   Adj. R-squared (uncentered):              0.857
Method:                 Least Squares   F-statistic:                              1282.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:22   Log-Likelihood:                         -4711.0
No. Observations:                1288   AIC:                                      9434.
Df Residuals:                    1282   BIC:                                      9465.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
PIB (mil R$)_1000               4.947e-05   9.83e-06      5.033      0.000    3.02e-05    6.88e-05
Profissionais_Privados_Prop        8.1259      1.197      6.791      0.000       5.778      10.473
Psi_valor_medio_1000               0.0728      0.001     60.425      0.000       0.070       0.075
Estabelecimentos_Privados_1000    -1.0410      0.409     -2.544      0.011      -1.844      -0.238
Atendimentos_valor_medio_1000      0.0001   4.73e-05      2.545      0.011    2.76e-05       0.000
Atendimentos_qtd_media_1000        0.0018      0.001      3.522      0.000       0.001       0.003
==============================================================================
Omnibus:                     1023.893   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22315.308
Skew:                           3.594   Prob(JB):                         0.00
Kurtosis:                      22.082   Cond. No.                     2.03e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.03e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
predict_tst = model_prop_out.predict(X_tst_prop_out.filter(cols))
predict_tr = model_prop_out.predict(X_tr_prop_out.filter(cols))
r2_score(y_tst_prop_out, predict_tst)

0.7356717209884249

In [ ]:
mape(y_tr_prop_out, predict_tr), mape(y_tst_prop_out, predict_tst)

(inf, inf)

Para o grupo 4.b. (base com variáveis de valores proporcionais e cidades com alta concentração de outliers), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.857 em treino e 0.735 em teste. Porém, observando a métrica MAPE, temos resultados bastante insatisfatórios.

Foram utilizadas as variáveis:
1. Estabelecimentos_Privados_1000
2. Atendimentos_qtd_media_1000
3. Psi_valor_medio_1000
4. Profissionais_Privados_Prop
5. PIB (mil R$)_1000
6. Atendimentos_valor_medio_1000

###Base b., Valores Normalizados



In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['Masculino_1000',
           'Estabelecimentos_Total_1000',
           '60+ anos_1000',
           'Estabelecimentos_Publicos_Prop',
           'Profissionais_Privados_1000',
           'Profissionais_Privados_Prop',
           '0 a 14 anos_1000',
           '15 a 29 anos_1000',
           'Profissionais_Total_1000',
           'Estabelecimentos_Psi_Prop',
           'Estabelecimentos_Atend_1000',
           'Estabelecimentos_Psi_1000']
cols = list((set(X_tr_prop_out_std.columns) - set(ids)) - set(removed))
X = X_tr_prop_out_std.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
7,Estabelecimentos_Atend_Prop,1.188167
4,Psi_valor_medio_1000,1.195887
9,Contribuição para o PIB estadual %,1.259903
15,Atendimentos_qtd_media_1000,1.341107
13,Atendimentos_valor_medio_1000,1.350282
5,Feminino_1000,1.432565
1,Profissionais_Publicos_1000,1.911986
8,Estabelecimentos_Publicos_1000,2.350201
12,Renda média em Salários mínimos,2.409724
3,Profissionais_Publicos_Prop,2.428870


In [ ]:
removed.append('Contribuição para o PIB estadual %')
removed.append('PIB (mil R$)_1000')
removed.append('Estabelecimentos_Privados_Prop')
removed.append('Estabelecimentos_Publicos_1000')
removed.append('Atendimentos_valor_medio_1000')
removed.append('Estabelecimentos_Atend_Prop')
removed.append('Assalariados_1000')

cols = list((set(X_tr_prop_out_std.columns) - set(ids)) - set(removed))
X = X_tr_prop_out_std.filter(cols).copy()

#intercepto removido com p-valor de 0.753
#X = sm.add_constant(X)

model_prop_out_std = sm.OLS(y_tr_prop_out_std, X).fit()

model_prop_out_std.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Psi_qtd_media_1000   R-squared (uncentered):                   0.803
Model:                            OLS   Adj. R-squared (uncentered):              0.801
Method:                 Least Squares   F-statistic:                              578.5
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:44:25   Log-Likelihood:                         -784.46
No. Observations:                1288   AIC:                                      1587.
Df Residuals:                    1279   BIC:                                      1633.
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
30 a 59 anos_1000                   0.0604      0.022      2.701      0.007       0.017       0.104
Profissionais_Publicos_1000         0.1395      0.015      9.494      0.000       0.111       0.168
Profissionais_Publicos_Prop        -0.0534      0.018     -2.952      0.003      -0.089      -0.018
Psi_valor_medio_1000                0.8044      0.013     60.715      0.000       0.778       0.830
Feminino_1000                      -0.0449      0.013     -3.559      0.000      -0.070      -0.020
Renda média (R$)                    0.0938      0.021      4.538      0.000       0.053       0.134
Estabelecimentos_Privados_1000     -0.0369      0.016     -2.334      0.020      -0.068      -0.006
Renda média em Salários mínimos    -0.0457      0.015     -2.983      0.003      -0.076      -0.016
Atendimentos_qtd_media_1000         0.0591      0.014      4.299      0.000       0.032       0.086
==============================================================================
Omnibus:                      974.119   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23534.282
Skew:                           3.283   Prob(JB):                         0.00
Kurtosis:                      22.885   Cond. No.                         3.66
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
predict_tst = model_prop_out_std.predict(X_tst_prop_out_std.filter(cols))
predict_tr = model_prop_out_std.predict(X_tr_prop_out_std.filter(cols))
r2_score(y_tst_prop_out_std, predict_tst)

0.7825797368959378

In [ ]:
mape(y_tr_prop_out_std, predict_tr), mape(y_tst_prop_out_std, predict_tst)

(1.2895178038194544, 1.1347991038734335)

Para o grupo 4.b. (base com variáveis de valores normalizados e cidades com alta concentração de outliers), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustado de 0.801 em treino e 0.782 em teste. Porém, observando a métrica MAPE, temos resultados insatisfatórios - ainda que melhores que a modelagem com valores proporcionais.

Foram utilizadas as variáveis:
1. Estabelecimentos_Privados_1000
2. Profissionais_Publicos_1000
3. 30 a 59 anos_1000
4. Atendimentos_qtd_media_1000
5. Renda média (R$)
6. Profissionais_Publicos_Prop
7. Psi_valor_medio_1000
8. Feminino_1000
9. Renda média em Salários mínimos

#Grupo 5 - Bases totais geral (a) e normalizada (b), target absoluta

###Base a., Base geral

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['0 a 14 anos',
           'Feminino',
           'Profissionais_Publicos',
           'Estabelecimentos_Publicos',
           'Estabelecimentos_Privados_1000',
           'Profissionais_Publicos_1000',
           'Feminino_1000',
           '60+ anos_1000',
           'Pop_Total',
           'Masculino',
           '30 a 59 anos',
           'Estabelecimentos_Total',
           'Estabelecimentos_Atend_Prop',
           'Estabelecimentos_Atend',
           '30 a 59 anos_1000',
           'Profissionais_Total',
           'Estabelecimentos_Psi_Prop',
           'Masculino_1000',
           '60+ anos',
           'Assalariados',
           'Contribuição para o PIB estadual %',
           '15 a 29 anos_1000',
           'Estabelecimentos_Atend_1000',
           'Estabelecimentos_Psi',
           'Atendimentos_qtd_media',
           '0 a 14 anos_1000',
           '15 a 29 anos',
           'PIB (mil R$)',
           'Renda média em Salários mínimos',
           'Estabelecimentos_Total_1000',
           'Atendimentos_valor_medio',
           'Profissionais_Publicos_Prop',
           'Renda média (R$)',
           'Estabelecimentos_Psi_1000',
           'Estabelecimentos_Privados',
           'Profissionais_Privados_Prop']
cols = list((set(X_tr_tot.columns) - set(ids)) - set(removed))
X = X_tr_tot.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
5,Psi_valor_medio_1000,1.639352
11,Atendimentos_qtd_media_1000,1.754339
9,Atendimentos_valor_medio_1000,1.783722
3,Profissionais_Privados_1000,2.515901
10,PIB (mil R$)_1000,2.846687
4,Profissionais_Privados,2.864080
7,Estabelecimentos_Privados_Prop,3.049434
2,Psi_valor_medio,3.451464
1,Assalariados_1000,3.922897
0,Profissionais_Total_1000,4.416013


In [ ]:
removed.append('Estabelecimentos_Privados_Prop')
removed.append('Estabelecimentos_Publicos_Prop')
removed.append('Atendimentos_qtd_media_1000')
removed.append('Estabelecimentos_Publicos_1000')
removed.append('Atendimentos_valor_medio_1000')

cols = list((set(X_tr_tot.columns) - set(ids)) - set(removed))
X = X_tr_tot.filter(cols).copy()

#intercepto removido com p-valor de 0.275
#X = sm.add_constant(X)

model_tot = sm.OLS(y_tr_tot, X).fit()

model_tot.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Psi_qtd_media   R-squared (uncentered):                   0.905
Model:                            OLS   Adj. R-squared (uncentered):              0.905
Method:                 Least Squares   F-statistic:                              9808.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        21:59:45   Log-Likelihood:                         -63132.
No. Observations:                7224   AIC:                                  1.263e+05
Df Residuals:                    7217   BIC:                                  1.263e+05
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
PIB (mil R$)_1000               0.0029      0.001      3.244      0.001       0.001       0.005
Assalariados_1000              -0.2757      0.131     -2.104      0.035      -0.533      -0.019
Psi_valor_medio                 0.0959      0.001     72.637      0.000       0.093       0.099
Profissionais_Privados_1000 -2387.1617    180.653    -13.214      0.000   -2741.295   -2033.028
Profissionais_Privados         35.7186      0.398     89.815      0.000      34.939      36.498
Psi_valor_medio_1000           -0.5764      0.112     -5.167      0.000      -0.795      -0.358
Profissionais_Total_1000      357.3836     85.869      4.162      0.000     189.056     525.711
==============================================================================
Omnibus:                     6028.806   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         19504913.766
Skew:                           2.484   Prob(JB):                         0.00
Kurtosis:                     257.511   Cond. No.                     3.52e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 3.52e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
predict_tst = model_tot.predict(X_tst_tot.filter(cols))
predict_tr = model_tot.predict(X_tr_tot.filter(cols))
r2_score(y_tst_tot, predict_tst)

0.8171253016682987

In [ ]:
mape(y_tr_tot, predict_tr), mape(y_tst_tot, predict_tst)

(inf, inf)

Para o grupo 5.a. (base total com variáveis de valores regulares), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustado de 0.905 em treino e 0.817 em teste. Porém, observando a métrica MAPE, temos resultados insatisfatórios.

Foram utilizadas as variáveis:
1. PIB (mil R$)_1000
2. Assalariados_1000
3. Psi_valor_medio
4. Profissionais_Privados_1000
5. Profissionais_Privados
6. Psi_valor_medio_1000
7. Profissionais_Total_1000

###Base b., Valores Normalizados

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['0 a 14 anos_1000',
           'Profissionais_Publicos',
           '15 a 29 anos',
           'Estabelecimentos_Total_1000',
           'Pop_Total',
           'Estabelecimentos_Total',
           'Masculino_1000',
           'Profissionais_Privados_1000',
           '30 a 59 anos',
           'Estabelecimentos_Psi_Prop',
           'Feminino',
           'Masculino',
           'Estabelecimentos_Atend',
           'Profissionais_Total',
           'Estabelecimentos_Privados_Prop',
           '60+ anos',
           'Assalariados',
           '0 a 14 anos',
           'Estabelecimentos_Psi',
           'Estabelecimentos_Atend_1000',
           'Contribuição para o PIB estadual %',
           'Atendimentos_valor_medio',
           'PIB (mil R$)',
           'Estabelecimentos_Publicos',
           'Estabelecimentos_Privados',
           'Profissionais_Total_1000',
           '15 a 29 anos_1000']
cols = list((set(X_tr_tot_std.columns) - set(ids)) - set(removed))
X = X_tr_tot_std.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
20,Atendimentos_qtd_media_1000,1.045757
14,Atendimentos_valor_medio_1000,1.303827
8,Estabelecimentos_Atend_Prop,1.323486
7,Assalariados_1000,1.525852
3,Psi_valor_medio_1000,1.583162
9,Profissionais_Publicos_1000,1.743928
18,Feminino_1000,1.791037
6,PIB (mil R$)_1000,1.834687
19,Renda média em Salários mínimos,1.995067
10,Profissionais_Publicos_Prop,2.022801


In [ ]:
removed.append('Profissionais_Publicos_Prop')
removed.append('Estabelecimentos_Atend_Prop')
removed.append('30 a 59 anos_1000')
removed.append('PIB (mil R$)_1000')
removed.append('Estabelecimentos_Publicos_1000')
removed.append('Feminino_1000')
removed.append('Assalariados_1000')

cols = list((set(X_tr_tot_std.columns) - set(ids)) - set(removed))
X = X_tr_tot_std.filter(cols).copy()

#intercepto removido com p-valor de 0.804
#X = sm.add_constant(X)

model_tot_std = sm.OLS(y_tr_tot_std, X).fit()

model_tot_std.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Psi_qtd_media   R-squared (uncentered):                   0.906
Model:                            OLS   Adj. R-squared (uncentered):              0.906
Method:                 Least Squares   F-statistic:                              4977.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        22:15:27   Log-Likelihood:                         -2084.7
No. Observations:                7224   AIC:                                      4197.
Df Residuals:                    7210   BIC:                                      4294.
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Estabelecimentos_Psi_1000          -0.0245      0.007     -3.644      0.000      -0.038      -0.011
Profissionais_Publicos_1000         0.0123      0.004      2.959      0.003       0.004       0.020
Profissionais_Privados_Prop        -0.0396      0.005     -7.318      0.000      -0.050      -0.029
Psi_valor_medio                     0.5362      0.007     74.237      0.000       0.522       0.550
Profissionais_Privados              0.5983      0.008     79.347      0.000       0.584       0.613
Psi_valor_medio_1000               -0.0426      0.005     -8.898      0.000      -0.052      -0.033
Estabelecimentos_Publicos_Prop     -0.0476      0.007     -6.402      0.000      -0.062      -0.033
Atendimentos_qtd_media             -0.1249      0.008    -15.653      0.000      -0.141      -0.109
Renda média (R$)                    0.0314      0.005      5.957      0.000       0.021       0.042
Estabelecimentos_Privados_1000     -0.0360      0.008     -4.590      0.000      -0.051      -0.021
Renda média em Salários mínimos    -0.0249      0.005     -4.993      0.000      -0.035      -0.015
60+ anos_1000                      -0.0153      0.005     -3.273      0.001      -0.025      -0.006
Atendimentos_valor_medio_1000      -0.0108      0.004     -2.600      0.009      -0.019      -0.003
Atendimentos_qtd_media_1000         0.0102      0.004      2.411      0.016       0.002       0.019
==============================================================================
Omnibus:                     6012.157   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         11523973.196
Skew:                           2.581   Prob(JB):                         0.00
Kurtosis:                     198.599   Cond. No.                         5.52
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
predict_tst = model_tot_std.predict(X_tst_tot_std.filter(cols))
predict_tr = model_tot_std.predict(X_tr_tot_std.filter(cols))
r2_score(y_tst_tot_std, predict_tst)

0.8223338531111059

In [ ]:
mape(y_tr_tot_std, predict_tr), mape(y_tst_tot_std, predict_tst)

(1.0835133744265113, 1.0330444131597578)

Para o grupo 5.b. (base total com variáveis de valores normalizados), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustado de 0.906 em treino e 0.822 em teste. Porém, observando a métrica MAPE, temos resultados insatisfatórios - ainda que melhores que a modelagem com valores regulares.

Foram utilizadas as variáveis:
1. Estabelecimentos_Psi_1000
2. Profissionais_Publicos_1000
3. Profissionais_Privados_Prop
4. Psi_valor_medio
5. Profissionais_Privados
6. Psi_valor_medio_1000
7. Estabelecimentos_Publicos_Prop
8. Atendimentos_qtd_media
9. Renda média (R$)
10. Estabelecimentos_Privados_1000
11. Renda média em Salários mínimos
12. 60+ anos_1000
13. Atendimentos_valor_medio_1000
14. Atendimentos_qtd_media_1000

#Grupo 6 - Bases totais geral (a) e normalizada (b), target proporcional

###Base a., Base geral

In [ ]:
#Lista de removidos de acordo com os scores indicativos de multicolinearidade
removed = ['0 a 14 anos',
           'Feminino',
           'Profissionais_Publicos',
           'Estabelecimentos_Publicos',
           'Estabelecimentos_Privados_1000',
           'Profissionais_Publicos_1000',
           'Feminino_1000',
           '60+ anos_1000',
           'Pop_Total',
           'Masculino',
           '30 a 59 anos',
           'Estabelecimentos_Total',
           'Estabelecimentos_Atend_Prop',
           'Estabelecimentos_Atend',
           '30 a 59 anos_1000',
           'Profissionais_Total',
           'Estabelecimentos_Psi_Prop',
           'Masculino_1000',
           '60+ anos',
           'Assalariados',
           'Contribuição para o PIB estadual %',
           '15 a 29 anos_1000',
           'Estabelecimentos_Atend_1000',
           'Estabelecimentos_Psi',
           'Atendimentos_qtd_media',
           '0 a 14 anos_1000',
           '15 a 29 anos',
           'PIB (mil R$)',
           'Renda média em Salários mínimos',
           'Estabelecimentos_Total_1000',
           'Atendimentos_valor_medio',
           'Profissionais_Publicos_Prop',
           'Renda média (R$)',
           'Estabelecimentos_Psi_1000',
           'Estabelecimentos_Privados',
           'Profissionais_Privados_Prop']
cols = list((set(X_tr_1000_tot.columns) - set(ids)) - set(removed))
X = X_tr_1000_tot.filter(cols).copy()

vif_data = pd.DataFrame()
temp = list(X.columns)
vif_data["feature"] = temp
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(temp))]

vif_data.sort_values(by='VIF')

,feature,VIF
5,Psi_valor_medio_1000,1.639352
11,Atendimentos_qtd_media_1000,1.754339
9,Atendimentos_valor_medio_1000,1.783722
3,Profissionais_Privados_1000,2.515901
10,PIB (mil R$)_1000,2.846687
4,Profissionais_Privados,2.864080
7,Estabelecimentos_Privados_Prop,3.049434
2,Psi_valor_medio,3.451464
1,Assalariados_1000,3.922897
0,Profissionais_Total_1000,4.416013


In [ ]:
removed.append('Estabelecimentos_Privados_Prop')
removed.append('Profissionais_Privados_1000')
removed.append('Estabelecimentos_Publicos_Prop')
removed.append('Atendimentos_valor_medio_1000')
removed.append('Estabelecimentos_Publicos_1000')
removed.append('Psi_valor_medio')
removed.append('Profissionais_Privados')
removed.append('Assalariados_1000')
removed.append('PIB (mil R$)_1000')

cols = list((set(X_tr_1000_tot.columns) - set(ids)) - set(removed))
X = X_tr_1000_tot.filter(cols).copy()

#intercepto removido com p-valor de 0.247
#X = sm.add_constant(X)

model_tot_1000 = sm.OLS(y_tr_1000_tot, X).fit()

model_tot_1000.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Psi_qtd_media_1000   R-squared (uncentered):                   0.456
Model:                            OLS   Adj. R-squared (uncentered):              0.456
Method:                 Least Squares   F-statistic:                              2017.
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                        0.00
Time:                        22:32:44   Log-Likelihood:                         -32270.
No. Observations:                7224   AIC:                                  6.455e+04
Df Residuals:                    7221   BIC:                                  6.457e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Atendimentos_qtd_media_1000     0.0004      0.000      2.760      0.006       0.000       0.001
Profissionais_Total_1000       11.3928      0.866     13.152      0.000       9.695      13.091
Psi_valor_medio_1000            0.0855      0.001     65.933      0.000       0.083       0.088
==============================================================================
Omnibus:                    18972.986   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        412668749.882
Skew:                          30.785   Prob(JB):                         0.00
Kurtosis:                    1172.274   Cond. No.                     6.80e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 6.8e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Para o grupo 6.a. (base total com variáveis de valores regulares), após exclusão de variáveis multicolineares e avaliação de p-valores, obtivemos modelo de regressão linear com r² ajustadio de 0.456 em treino - baixa explicabilidade da variável target.

Foram utilizadas apenas as variáveis:
1. Atendimentos_qtd_media_1000
2. Profissionais_Total_1000
3. Psi_valor_medio_1000